In [1]:
#install.packages('partykit',repos='http://cran.us.r-project.org')

In [2]:
library(data.table)
library(ggplot2)
library(partykit)
library(plyr)

Warning message:
"package 'ggplot2' was built under R version 3.5.3"Warning message:
"package 'partykit' was built under R version 3.5.3"Loading required package: grid
Loading required package: libcoin
Warning message:
"package 'libcoin' was built under R version 3.5.3"Loading required package: mvtnorm
Warning message:
"package 'mvtnorm' was built under R version 3.5.2"Warning message:
"package 'plyr' was built under R version 3.5.3"

In [3]:
Data=read.csv('R_Training_Counting.csv')
Data[Data[,'NumPax']<0,'NumPax']=0

In [4]:
class(eq.mob <- NumPax ~ 1 + Dprio | dday + dtime + Direction + month)  

curve.model <- glmtree(
        eq.mob, 
        data = Data,
        family = poisson, 
        alpha = 0.01,
        bonferroni = TRUE,
        verbose = TRUE,
        prune = "BIC",
        minsize = 10,
        breakties = TRUE,
        restart = TRUE,
        maxdepth=5)

[1] "formula"


-- Node 1 ---------------------------------
Number of observations: 625245

Parameter instability tests:
                   dday        dtime  Direction        month
statistic  5.876573e+02 1.286510e+02 7.64751478 4.182691e+02
p.value   1.830726e-117 6.689004e-27 0.08456036 6.093755e-93

Best splitting variable: dday
Perform split? yes
Selected split: Friday, Saturday, Sunday | Monday, Thursday, Tuesday, Wednesday

-- Node 2 ---------------------------------
Number of observations: 204765

Parameter instability tests:
                  dday        dtime    Direction        month
statistic 9.995697e+01 1.793245e+02 4.677932e+01 1.733772e+02
p.value   4.018522e-20 2.139583e-38 2.780100e-10 4.810858e-37

Best splitting variable: dtime
Perform split? yes
Selected split: <= 465 | > 465

-- Node 3 ---------------------------------
Number of observations: 35405

Parameter instability tests:
                dday        dtime Direction        month
statistic 9.72497245 5.139063e+01         0 2

In [5]:
forecast_bookings=as.numeric(predict(curve.model,newdata=Data,type='response'))
forecast_node=as.numeric(predict(curve.model,newdata=Data,type='node'))
Data=cbind(Data,forecast_bookings,forecast_node)
write.csv(Data,'R_Output_Training.csv',row.names=FALSE)

In [6]:
Forecast_Data=read.csv('R_Test_Counting.csv')
forecast_bookings=as.numeric(predict(curve.model,newdata=Forecast_Data,type='response'))
forecast_node=as.numeric(predict(curve.model,newdata=Forecast_Data,type='node'))
Forecast_Data=cbind(Forecast_Data,forecast_bookings,forecast_node)
write.csv(Forecast_Data,'R_Output_Test.csv',row.names=FALSE)